### Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [48]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import time

In [61]:
# Добавил в поиск слова из текущих постов для проверки
KEYWORDS = ['SQLite3', 'python', 'парсинг', 'СУБД']

In [73]:
request_ = rq.get('https://habr.com/ru/all/')
soup = BeautifulSoup(request_.text)

In [72]:
# Находим блоки со статьями 
blocks_articles = soup.find_all('article', class_='tm-articles-list__item')

In [64]:
# Находим ссылки на статью на полный текст статьи 
atricle_links = list(map(lambda x: x.find('a', class_='tm-article-snippet__readmore').get('href'), blocks_articles ))
links = list(map(lambda x: 'https://habr.com' + x, atricle_links))

In [68]:
fresh_news_habr = pd.DataFrame()
for line in links:
    soup = BeautifulSoup(rq.get(line).text)
    time.sleep(0.3)
    text = soup.find('div', xmlns='http://www.w3.org/1999/xhtml').get_text()
    for word in KEYWORDS:
        if word in text:
            date = pd.to_datetime(soup.find('time').get('datetime'))
            title = soup.find('h1', class_='tm-article-snippet__title tm-article-snippet__title_h1').text
            row = {'date': date, 'link': line, 'title': title,'article_text': text}
            fresh_news_habr = pd.concat([fresh_news_habr, pd.DataFrame([row])])

In [69]:
fresh_news_habr

,date,link,title,article_text
0,2021-07-06 13:03:01+00:00,https://habr.com/ru/company/ruvds/blog/566098/,Лампа для слежения за фазами Луны на Raspberry...,"Лампа для слежения за фазами Луны, об изготовл..."
0,2021-07-06 12:00:02+00:00,https://habr.com/ru/company/macloud/blog/566396/,Интервью с создателем SQLite (часть 1): истори...,\nАдам: Привет и добро пожаловать в CoRecursiv...


In [70]:
# Код без keywords для проверки

fresh_news_habr = pd.DataFrame()
for index, line in enumerate(links, start=1):
    soup = BeautifulSoup(rq.get(line).text)
    time.sleep(0.3)
    text = soup.find('div', xmlns='http://www.w3.org/1999/xhtml').get_text()
    date = pd.to_datetime(soup.find('time').get('datetime'))
    title = soup.find('h1', class_='tm-article-snippet__title tm-article-snippet__title_h1').text
    row = {'№':index, 'date': date, 'link': line, 'title': title,'article_text': text.strip()}
    fresh_news_habr = pd.concat([fresh_news_habr, pd.DataFrame([row])])

In [71]:
fresh_news_habr


,№,date,link,title,article_text
0,1,2021-07-06 15:18:19+00:00,https://habr.com/ru/post/566440/,И снова про MS Excel,Однажды в моем инстаграме появились сразу неск...
0,2,2021-07-06 14:45:02+00:00,https://habr.com/ru/company/englishdom/blog/56...,7 фразовых глаголов с take и 30 их значений. Н...,В одном из прошлых материалов мы рассказали ва...
0,3,2021-07-06 14:27:24+00:00,https://habr.com/ru/post/566436/,"Фантасты, которых стоит почитать любителям анг...","Английский юмор — словосочетание, которое инту..."
0,4,2021-07-06 14:14:20+00:00,https://habr.com/ru/company/rostelecom/blog/56...,Краткое руководство по LXC в ОС Эльбрус,Не так давно Ростелеком объявил о создании цен...
0,5,2021-07-06 13:53:15+00:00,https://habr.com/ru/post/566426/,Как мы боролись за байты на фронтенде. Усмиряе...,Вопрос производительности все чаще обсуждается...
0,6,2021-07-06 13:03:01+00:00,https://habr.com/ru/company/ruvds/blog/566098/,Лампа для слежения за фазами Луны на Raspberry...,"Лампа для слежения за фазами Луны, об изготовл..."
0,7,2021-07-06 12:44:59+00:00,https://habr.com/ru/post/566394/,Валидация в приложении на PHP (часть 1 — валид...,"Как по мне достаточно важный, хотя и холиварны..."
0,8,2021-07-06 12:35:02+00:00,https://habr.com/ru/company/dcmiran/blog/566326/,Как выглядит современный дата-центр изнутри,«Миран» пришел на рынок коммерческих ЦОДов в 2...
0,9,2021-07-06 12:27:57+00:00,https://habr.com/ru/post/566410/,Как я сделал свой трекер мечты и понял цену вр...,Основной посыл этого поста - рассказать о разв...
0,10,2021-07-06 12:18:36+00:00,https://habr.com/ru/company/2gis/blog/566178/,За что мы (не) любим нативную мобильную разраб...,Привет! Вечером 8 июля мы проведем открытый он...
